In [17]:
import pandas as pd
import numpy as np
from google.colab import files
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from joblib import dump
from google.colab import drive

In [6]:
uploaded = files.upload()

df = pd.read_csv(next(iter(uploaded)))
df.head()

Saving revised_data_for_yoga_program_ML_training.csv to revised_data_for_yoga_program_ML_training.csv


,pose_name,nose_x,nose_y,nose_z,left_shoulder_x,left_shoulder_y,left_shoulder_z,right_shoulder_x,right_shoulder_y,right_shoulder_z,...,left_shoulder_hip_knee_angle,right_shoulder_hip_knee_angle,left_hip_knee_ankle_angle,right_hip_knee_ankle_angle,left_shoulder_elbow_wrist_angle,right_shoulder_elbow_wrist_angle,left_hip_shoulder_elbow_angle,right_hip_shoulder_elbow_angle,left_head_shoulder_hip_angle,right_head_shoulder_hip_angle
0,cat pose,-0.516661,0.129909,-0.040134,-0.456642,0.087930,-0.150298,-0.559500,-0.048387,0.167710,...,90.383228,95.917025,94.432062,91.495575,58.989127,47.994986,80.646952,112.963827,69.146791,106.356016
1,cat pose,-0.439104,0.075872,-0.051650,-0.420443,-0.063529,-0.126287,-0.493098,-0.071897,0.195785,...,78.669241,84.834516,94.975611,85.063283,50.151370,29.404093,98.053367,114.593382,95.507592,116.206253
2,cat pose,0.416784,0.036046,-0.071157,0.431970,-0.062432,0.122458,0.469218,-0.058143,-0.100613,...,75.142867,91.846252,82.242260,102.820151,11.712166,36.841794,83.740180,104.608740,98.904615,124.027859
3,cat pose,0.392603,0.035017,-0.025522,0.418557,-0.052292,0.176311,0.476803,-0.051133,-0.068131,...,73.774703,85.272324,81.884197,99.688948,10.293739,38.302575,83.449668,109.410328,107.042943,133.637379
4,cat pose,-0.400933,0.082890,-0.103624,-0.404127,-0.033562,-0.184631,-0.523160,-0.068294,0.120899,...,80.949129,89.740441,96.669858,95.909843,46.580593,24.952177,88.362066,105.823007,104.459171,121.998540


In [7]:
class_name = df['pose_name'].unique()
print(class_name)

['cat pose' 'chair pose' 'corpse pose' 'cow pose' 'downward dog pose'
 'flat back pose' 'high lunge pose' 'knee to elbow plank pose'
 'knees to chest pose' 'low lunge pose' 'mountain pose'
 'runners lunge twist pose' 'seated spinal twist pose'
 'side plank yoga pose' 'standing forward bend pose' 'tabletop pose'
 'three legged dog pose' 'tip toe pose' 'tree pose' 'upper plank pose']


In [8]:
#assigning the values of the df to a numpy array
data=df.values

In [9]:
num_rows = data.shape[0]
num_cols = data.shape[1]

print(num_rows)
print(num_cols)

1496
50


In [10]:
#features= pose landmarks, target= pose name
features = data[:, 1:]
target = data[:, 0]

In [11]:
#splitting data
features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.2,random_state=42, shuffle= True)

In [12]:
#Model #4 Random Forest Classifier
model_RFC = RandomForestClassifier(n_estimators=100, random_state=35)
model_RFC.fit(features_train, target_train)

RandomForestClassifier(random_state=35)

In [13]:
prediction_RFC= model_RFC.predict(features_test)
print(accuracy_score(target_test, prediction_RFC)*100)

95.33333333333334


In [14]:
#for analyzing the results
probability_scores = model_RFC.predict_proba(features_test)
RFC_model_results = []
for i in range(len(prediction_RFC)):
  predicted_val = prediction_RFC[i]
  label_val = target_test[i]
  arr = probability_scores[i]
  formatted_percentage_output = [round(value * 100, 2) for value in arr]
  formatted_percentage_output.append(predicted_val)
  formatted_percentage_output.append(label_val)
  if predicted_val == label_val:
    formatted_percentage_output.append('YES')
  else:
    formatted_percentage_output.append('NO')

  RFC_model_results.append(formatted_percentage_output)

In [15]:
#for analyzing the results
RFC_df = pd.DataFrame(RFC_model_results, columns= ['cat pose','chair pose','corpse pose','cow pose','downward dog pose','flat back pose','high lunge pose',\
                                                   'knee to elbow plank pose','knees to chest pose','low lunge pose','mountain pose','runners lunge twist pose',\
                                                   'seated spinal twist pose','side plank yoga pose','standing forward bend pose','tabletop pose','three legged dog pose',\
                                                   'tip toe pose','tree pose','upper plank pose','predicted label','true label','accurate'])
RFC_df.to_csv('RFC_model_results.csv', index=False)

files.download('/content/RFC_model_results.csv')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [20]:
#looping through incorrect predictions
for i in range(len(prediction_RFC)):
  if target_test[i] != prediction_RFC[i]:
    print(target_test[i],prediction_RFC[i])

seated spinal twist pose tip toe pose
low lunge pose high lunge pose
runners lunge twist pose side plank yoga pose
high lunge pose low lunge pose
cow pose tabletop pose
standing forward bend pose flat back pose
high lunge pose low lunge pose
cow pose tabletop pose
high lunge pose low lunge pose
standing forward bend pose flat back pose
high lunge pose low lunge pose
high lunge pose low lunge pose
seated spinal twist pose tip toe pose
high lunge pose low lunge pose


In [19]:
# saving model
drive.mount('/content/drive')
dump(model_RFC, '/content/drive/My Drive/model4_RFC.joblib')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


['/content/drive/My Drive/model4_RFC.joblib']